In [1]:
using POMDPs, POMDPTools, QuickPOMDPs, MCTS, DiscreteValueIteration, POMDPSimulators, POMDPModels, POMDPPolicies
using Distributions, Combinatorics, StaticArrays, D3Trees, Random
using FileIO, JLD2, TickTock

In [2]:
action=[(0,0), (0,1), (1,1)]

Number_level = 10;
#Transition_matrix = [0.86 0.14 0.0 0.0; 0.0 0.83 0.17 0.0;0.0 0.0 0.89 0.11;1.0 0.0 0.0 0.0];
fullname = "tm10";
# generate_transition_matrix(Number_level, fullname) 
Transition_matrix = load(fullname*".jld2","transition_matrix");
failure_penalty = -1000.0;
maintenance_penalty = -200.0;
setup_cost = -800.0;
system_penalty = - 2000.0;
normal_operation = 0.0;

In [3]:
# Note that the reward function here is only decided by component-wise independent knowledge, 
# i.e., we know nothing about other components when making decision for this component
function reward(s, a,s1=10, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair
    #    if s==Number_level
    #         #r=failure_penalty  + normal_operation  +  setup_cost/NumberUnits  + system_penalty/NumberUnits;
    #         r=system_penalty/limit
    #     return r
    #    end
    #     r=0
     if a==1 # don't replace component
        if Number_level>s
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r = normal_operation ;
         return r
        else
            r=system_penalty/NumberUnits;
            return r
        end
        end

    if a==2 # replace component
        if Number_level>s
            s=1
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r = maintenance_penalty+  setup_cost/NumberUnits;
         return r
        else
            s=1
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r= failure_penalty  +  setup_cost/NumberUnits;
        return r
        end
        end
    end


reward (generic function with 3 methods)

In [4]:
s=1:Number_level;

In [5]:
T=zeros(Number_level,Number_level,3);


In [6]:
((1:10)./10)

0.1:0.1:1.0

In [7]:
T[:,:,1].=Transition_matrix;

In [8]:
T[:,:,2].=Transition_matrix;

In [9]:
for j in 1:10
        T[j,:,2].=Transition_matrix[1,:];
        end
# for j in 1:10
#         T[j,:,3].=0.0
#     T[j,1,3]=1.0
#         end

In [10]:
T[10,:,1].=Transition_matrix[1,:]
T[10,:,2].=Transition_matrix[1,:]

10-element view(::Array{Float64, 3}, 10, :, 2) with eltype Float64:
 0.2695279572108137
 0.16860741623354286
 0.1306613209439592
 0.09857168678543618
 0.09171796107138079
 0.09134806335671229
 0.09048875946531719
 0.04832973229444604
 0.00607725632113932
 0.004669846317252394

In [11]:
function explore()
    c=[1/3,1/3,1/3]

     crd = Categorical(c);
     return rand(crd)
    end 
function generative(s, a, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair

r=0
    if a==1 # don't replace component
        if Number_level>s
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r = normal_operation ;
            return (sp=s, r=r)
        else
            r=0;
            return (sp=s, r=r)
        end
        end

    if a==2 # replace component
        if Number_level==s
            s=1
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r= failure_penalty;
            
            return (sp=s, r=r)
        end
        if s==1
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r= 0;
            return (sp=s, r=r)
        end
        s=1
        crd = Categorical(Transition_matrix[s,:]);
        s = rand(crd);
        r = maintenance_penalty;
        return (sp=s, r=r)
    end
    end

generative (generic function with 2 methods)

In [12]:
# the value iteration here is simply based on independent component-level knowledge
function ValueIteration(S, T, γ, p1=0.01, d=100)
    Q=zeros((Number_level,3))
#     print(Q)
    V= rand(Float64, (Number_level))*1000
    diff=0
    i=0
    
    

    
    p=[1 1 1 ]
    while i<=2000
#     while  i<=10000
    for s in 1:10
        Threads.@threads for a in 1:2
 
                Q[s,a]= (reward(s,a)) + γ*( sum(T[s,s1,a]*(V[s1]) for (s1) in 1:10 ))
    end
            V[s]=findmax(Q[s,1:2])[1]
        end
        i+=1
        end
    return Q,V
    end

ValueIteration (generic function with 3 methods)

In [13]:


function decison(Q, s,n=0.2)
   a_final=[]
   if true
        for i in 1:NumberUnits
            append!(a_final,argmax(a->Q[s[i],a], [1,2]))
            end
        return a_final
    end
end

decison (generic function with 2 methods)

In [14]:

using DataFrames
df= DataFrame(u=[],K=[],n=[],s=[],m=[],f=[],p=[],lamb=[],mean=[],std=[])
cost=[ [0	-200	-100	-1200 -2000],
    [0	-400	-100	-1200 -2000],
    [0	-600	-100	-1200 -2000],
    [0	-800	-100	-1200 -2000],
    [0	-1000	-100	-1200 -2000],
    [0	-1200	-100	-1200 -2000],
    [0	-1400	-100	-1200 -2000],
    [0	-1600	-100	-1200 -2000],
    [0	-1800	-100	-1200 -2000],
    [0	-2000	-100	-1200 -2000],
    [0  -2400   -100    -1200 -2000],
    [0  -2800   -100    -1200 -2000],
    [0  -3200   -100    -1200 -2000]
] 



for units in [60,50,40,30,25,20]
    for K in 2:2:convert(Int64, floor((units+1)/2))
        global NumberUnits=convert(Int64,units)
        global limit=convert(Int64, K)
    for c in cost
   print(units)  
   print("K : ")
    println(K)
    global normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty=c
     println([normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty])   
     for lambda in 1:1:1
        global  lamb=lambda
     Q,V=ValueIteration(s,T,.95,3000)
         println(isnan(Q[1,1]))
       if sum(isnan(Q[1,1]))>0.0
            continue
        end
tick()
trials=10000;
simsteps = 100;
results=zeros(trials)*0.1
Threads.@threads for h in 1:trials
s=repeat(1:1,NumberUnits)
r1=[]
a1=[]   
r=repeat(1:1,NumberUnits)*1.0
 for k in 1:simsteps
    a =decison(Q,s)
    r2=0.0
    s1=repeat(1:1,NumberUnits)
              
# Here we are calculating the "true" system-level rewards, following formulation (3) in original paper
# i.e., adding up reward of each component, and decide to add setup cost or not depending on system-level condition 
# In contrast, the policy we are using is obtained only at component level          
#     print(a)
global failed_components = 0
    for i in 1:NumberUnits
        if s[i] == Number_level
            failed_components += 1
         end
#             print(generative(s[i],a[i],234))
         s1[i],r[i]=generative(s[i],a[i],234);
          r2+=r[i]
        
            end 
#     println(r2)
#     println(s)

    if  (2 in a) 
        # if any replacement
            r2+=setup_cost
                    end
    if failed_components >= limit
        r2+=system_penalty
    end
    s=s1;
    append!(a1,a)
    append!(r1,r2)
end
rk=0
for i in 1:simsteps
    rk=0.95*rk+r1[simsteps-i+1]
end
#     println(rk)
    results[h]=rk
#     println(results)
end
println(units)
println(lambda)

println(mean(results))

println(std(results))
pushfirst!(df,[units,limit,normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty,lamb,mean(results),std(results)])
        
tock()
end
    println("-------------------------")
end
end
end
end

60K : 2
[0, -200, -100, -1200, -2000]
false


[ Info:  started timer at: 2024-03-14T07:32:42.558


60
1
-38468.66200979835
1150.962166577496
-------------------------
60K : 2
[0, -400, -100, -1200, -2000]


[ Info:           15.2573605s: 15 seconds, 257 milliseconds


false


...

false


[ Info:  started timer at: 2024-03-14T10:06:51.853


20
1
-23463.527748970737
3269.6945994240764
-------------------------


[ Info:            4.6141647s: 4 seconds, 614 milliseconds


LoadError: syntax: unexpected "end"

In [15]:
df

Row,u,K,n,s,m,f,p,lamb,mean,std
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,20.0,10.0,0.0,-3200.0,-100.0,-1200.0,-2000.0,1.0,-23463.5,3269.69
2,20.0,10.0,0.0,-2800.0,-100.0,-1200.0,-2000.0,1.0,-23497.8,3260.41
3,20.0,10.0,0.0,-2400.0,-100.0,-1200.0,-2000.0,1.0,-23529.1,3241.22
4,20.0,10.0,0.0,-2000.0,-100.0,-1200.0,-2000.0,1.0,-22950.5,3290.55
5,20.0,10.0,0.0,-1800.0,-100.0,-1200.0,-2000.0,1.0,-22973.0,3296.56
6,20.0,10.0,0.0,-1600.0,-100.0,-1200.0,-2000.0,1.0,-23379.0,3300.27
7,20.0,10.0,0.0,-1400.0,-100.0,-1200.0,-2000.0,1.0,-23501.3,3239.02
8,20.0,10.0,0.0,-1200.0,-100.0,-1200.0,-2000.0,1.0,-23479.3,3238.26
9,20.0,10.0,0.0,-1000.0,-100.0,-1200.0,-2000.0,1.0,-23529.9,3230.64


In [16]:
using CSV
CSV.write("Homogeneous Independent solution.csv", df)

"Homogeneous Independent solution.csv"

In [ ]:
#=  Q=ValueIteration(s,T,.95,3000)
for s in 1:Number_level
p=[exp(0.01*sum(Q[s,1])),exp(0.01*sum(Q[s,2])),exp(0.01*sum(Q[s,3]))]
    p./=sum(p)
    println(p)
end =#

In [ ]:
#= using Plots =#

In [ ]:
#= for i in [3,5,10,15,20,25,30,40,50,60]
   display( plot(df[df.Units.==i,:].lambda,df[df.Units.==i,:].mean, xtick=df[df.Units.==i,:].lambda, show = true,title=string(i),label=false, xaxis=:log10,rotation=40))
    plot(show=true)
    println(df[df.Units.==i,:][findmax(df[df.Units.==i,:].mean)[2],:])

end =#

In [ ]:
#= plot!()
 =#